<a href="https://colab.research.google.com/github/gmihaila/gtc2020_instructor_training/blob/master/sentiment_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Sanity Check

Click the _Runtime_ dropdown at the top of the page, then _Change Runtime Type_ and confirm the instance type is _GPU_.

Check the output of `!nvidia-smi` to make sure you've been allocated a Tesla T4 or P100.

https://colab.research.google.com/drive/1rY7Ln6rEE1pOlfSHCYOVaqt8OvDO35J0#forceEdit=true&sandboxMode=true&scrollTo=scfLT2i0MLyD

In [1]:
## VERSION CHECK [FACTORY RESET UNTIL GET AT LEAST 16280MiB GPU ]
!nvidia-smi
!nvcc -V
!python -V; pip -V

Mon Feb 10 22:28:51 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Installs

**Set up script installs**

1. Install most recent Miniconda release compatible with Google Colab's Python install  (3.6.7)
1. removes incompatible files
1. Install RAPIDS libraries
1. Set necessary environment variables
1. Copy RAPIDS .so files into current working directory, a workaround for conda/colab interactions
1. Recommend select v0.10
If running v0.11 or higher, updates pyarrow library to 0.15.x.

In [2]:
## SENTENCE TRANSFORMERS
!pip install git+https://github.com/arvkevi/kneed
!pip install git+https://github.com/UKPLab/sentence-transformers
!pip install -U ipykernel

# Install RAPIDS
!rm rapids-colab.sh
!wget -nc https://raw.githubusercontent.com/rapidsai/notebooks-contrib/890b04ed8687da6e3a100c81f449ff6f7b559956/utils/rapids-colab.sh
!bash rapids-colab.sh

import sys, os

dist_package_index = sys.path.index("/usr/local/lib/python3.6/dist-packages")
sys.path = sys.path[:dist_package_index] + ["/usr/local/lib/python3.6/site-packages"] + sys.path[dist_package_index:]
sys.path
if os.path.exists('update_pyarrow.py'): ## This file only exists if you're using RAPIDS version 0.11 or higher
  exec(open("update_pyarrow.py").read(), globals())

!pip install -U ipykernel

  Cloning https://github.com/arvkevi/kneed to /tmp/pip-req-build-po5hs7fj
  Running command git clone -q https://github.com/arvkevi/kneed /tmp/pip-req-build-po5hs7fj
  Created wheel for kneed: filename=kneed-0.5.1-py2.py3-none-any.whl size=10036 sha256=f096cea28022aae83aefbc02b3140d3a39241cb797d762f2d2e937433a3814a1
  Stored in directory: /tmp/pip-ephem-wheel-cache-ptqw75fi/wheels/5d/da/82/4f1e7e6e7daf1feced47f9cf350053b535a747737830d86ca0
Successfully built kneed
  Cloning https://github.com/UKPLab/sentence-transformers to /tmp/pip-req-build-oxjij8vk
  Running command git clone -q https://github.com/UKPLab/sentence-transformers /tmp/pip-req-build-oxjij8vk
     |████████████████████████████████| 450kB 3.4MB/s 
     |████████████████████████████████| 870kB 17.0MB/s 
     |████████████████████████████████| 1.0MB 20.5MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.5-cp36-none-any.whl size=69894 sha256=7b68b898160c7fe9d36626cf4631056ffbf494f5bbdf26d147ba

rm: cannot remove 'rapids-colab.sh': No such file or directory
--2020-02-10 22:29:18--  https://raw.githubusercontent.com/rapidsai/notebooks-contrib/890b04ed8687da6e3a100c81f449ff6f7b559956/utils/rapids-colab.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5286 (5.2K) [text/plain]
Saving to: ‘rapids-colab.sh’

rapids-colab.sh     100%[===================>]   5.16K  --.-KB/s    in 0s      

2020-02-10 22:29:18 (77.1 MB/s) - ‘rapids-colab.sh’ saved [5286/5286]

PLEASE READ
********************************************************************************************************
Colab v0.11+ Migration Notice:
 
There has been a NECESSARY Colab script code change for VERSION 0.11+ that MAY REQUIRE an update how you install RAPIDS into Colab!  
Not all Colab notebook

# Downloads

# Imports

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
# Import TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import tensorflow_datasets as tfds

os.environ["CUDA_VISIBLE_DEVICES"] = '0'
import numpy as np
from multiprocessing import Process, Pipe
from collections import Counter
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from kneed import KneeLocator
import seaborn as sns
import pandas as pd
from multiprocessing import Process, Pipe
from sentence_transformers import SentenceTransformer
import torch

PRETRAINED_MODEL = 'bert-base-nli-stsb-mean-tokens'

TensorFlow 2.x selected.


# Functions

In [0]:
def tf_dataset(dataset_name):
  builder = tfds.builder(dataset_name)
  builder.download_and_prepare()
  datasets = builder.as_dataset()

  movies_reviews = []
  sentiments_reviews = []
  for partition in ['train', 'test']:
    for instance in datasets[partition]:
      sentiment_review = tf.get_static_value(instance['label'])
      sentiments_reviews.append(sentiment_review)
      movie_review = tf.get_static_value(instance['text'])
      movie_review = movie_review.decode()
      movies_reviews.append(movie_review)
  return movies_reviews, sentiments_reviews

def context_embedding(text, name_of_model, batch_size=1000, use_device="cuda"):

  embed_model = SentenceTransformer(model_name_or_path=name_of_model,
                                    device=torch.device(use_device))

  encodings_text = embed_model.encode(sentences=text, 
                                          batch_size=batch_size, 
                                          show_progress_bar=True)
  return  encodings_text

# Dataset

In [6]:
movies_reviews, sentiments_reviews = tf_dataset(dataset_name='imdb_reviews')

print(movies_reviews[:2], sentiments_reviews[:2])
print(len(sentiments_reviews))

INFO:absl:No config specified, defaulting to first: imdb_reviews/plain_text
INFO:absl:Overwrite dataset info from restored data version.
INFO:absl:Reusing dataset imdb_reviews (/root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0)
INFO:absl:Constructing tf.data.Dataset for split None, from /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0


["This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", 'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was

In [5]:
movies_reviews_embeddings = context_embedding(text=movies_reviews, name_of_model=PRETRAINED_MODEL, batch_size=1000, use_device="cuda")

100%|██████████| 405M/405M [00:19<00:00, 21.0MB/s]
Batches:  86%|████████▌ | 43/50 [10:13<02:00, 17.26s/it]

KeyboardInterrupt: ignored

AttributeError: ignored

In [4]:
tf.config.list_logical_devices('GPU')

[LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [5]:
tf.config.set_visible_devices(tf.device('/CPU:0')) 
# tf.config.list_logical_devices('GPU')

ValueError: ignored

In [14]:
physical_devices = tf.config.list_physical_devices('GPU') 
try: 
  # Disable first GPU 
  tf.config.set_visible_devices(physical_devices[1:], 'GPU') 
  logical_devices = tf.config.list_logical_devices('GPU') 
  print(logical_devices)
  # Logical device was not created for first GPU 
  assert len(logical_devices) == len(physical_devices) - 1 
except: 
  # Invalid device or cannot modify virtual devices once initialized. 
  pass 

physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [0]:
tf.device('/CPU:0')

In [0]:
tf.config.set_visible_devices(
    devices,
    device_type=None
)


physical_devices = tf.config.list_physical_devices('GPU') 
try: 
  # Disable first GPU 
  tf.config.set_visible_devices(physical_devices[1:], 'GPU') 
  logical_devices = tf.config.list_logical_devices('GPU') 
  # Logical device was not created for first GPU 
  assert len(logical_devices) == len(physical_devices) - 1 
except: 
  # Invalid device or cannot modify virtual devices once initialized. 
  pass 
